In [1]:
# lstm autoencoder to recreate a timeseries
#gridsearch: https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
#https://machinelearningmastery.com/how-to-grid-search-deep-learning-models-for-time-series-forecasting/
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from tensorflow import keras
import pickle
import pyarrow.parquet as pq
from gensim.models import Word2Vec
import umap
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns
import pandas as pd
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from sklearn import preprocessing
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####

import time
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from functools import reduce
from ppca import PPCA
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
import json
from sklearn import metrics
from sklearn.decomposition import FastICA
from sklearn.metrics import pairwise_distances
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from pickle import load
from pickle import dump
import pickle
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import preprocessing
import scipy.cluster.hierarchy as shc
import scipy.stats as stats
import researchpy as rp
from keras.models import Model
from keras.layers import Dense, Input
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split  
from keras.layers import Input, Dense 
from keras.models import Model, Sequential 
from keras import regularizers 
import umap
from sklearn.cluster import DBSCAN
import hdbscan
from statistics import mean 

In [3]:
a=pq.read_table('Cohort/Metrics_LSTM.parquet').to_pandas()
a=a.sort_values(by=['data_structure','loss'])
#a=
a.loc[a['timesteps']==7,'data_structure']='per_day'
a=a.sort_values(by=['data_structure','loss'])
a

,data_structure,timesteps,n_features,layer_size_1,layer_size_2,activation_func,optimizer_func,loss_func,n_epochs,n_batch_size,loss
19,per_day,7,20,128,64,tanh,adam,mse,300,1024,0.203631
14,per_day,7,20,128,64,relu,adam,mse,300,1024,0.211849
22,per_day,7,20,100,50,relu,adam,mse,300,1024,0.216019
23,per_day,7,20,100,50,tanh,adam,mse,300,1024,0.284875
0,per_day,7,20,32,16,tanh,adam,mse,300,100,0.466053
13,per_day,7,20,32,16,relu,adam,mse,300,1024,0.759793
16,per_day,7,20,32,16,tanh,adam,mse,300,1024,0.908554
20,per_day,7,20,128,64,sigmoid,adam,mse,300,1024,1.399814
24,per_day,7,20,100,50,sigmoid,adam,mse,300,1024,1.596079
15,per_day,7,20,32,16,sigmoid,adam,mse,300,1024,2.100027


In [4]:
def load_dataset(per_day,embedding_method,embedding_size):
    if per_day: 
        df_name='Timeseries_per_patient_per_day_LSTM_Data_embedded_'
        data_structure='per_patient'
    else : 
        df_name='Timeseries_per_patient_LSTM_Data_embedded_'
        data_structure='per_patient'
    if embedding_method=='cbow':
        df_name=df_name+'cbow_dim{}_win5_mc0'.format(embedding_size)
    if embedding_method=='skipgram':
        df_name=df_name+'skipgram_dim{}_win5_mc0'.format(embedding_size)
    print(df_name)
    with open("Cohort/Time_Series/"+df_name+'.txt', "rb") as fp:   # Unpickling
        data = pickle.load(fp)
    data_sample= data[:-2000]
    return data,data_sample, df_name, data_structure

In [5]:
def load_lstm_representation(data_structure,timesteps,n_features,layer_size_1,layer_size_2,activation_func,optimizer_func,loss_func,n_epochs,n_batch_size,df_name,data):
    #create experiment Name to Load model 
    #experiment_name=df_name+'_'+data_structure+str(layer_size_1)+'_'+str(layer_size_2)+'_'+activation_func+'_'+optimizer_func+'_'+loss_func+'_'+str(n_epochs)+'_'+str(n_batch_size)
    experiment_name=df_name+'_'+data_structure+str(layer_size_1)+'_'+str(layer_size_2)+'_'+activation_func+'_'+optimizer_func+'_'+loss_func+'_'+str(n_epochs)+'_'+str(n_batch_size)


    
    model = keras.models.load_model('Cohort/LSTM_Models/'+experiment_name)
    hidden_representation = Sequential() 
    hidden_representation.add(model.layers[0]) 
    hidden_representation.add(model.layers[1])
    dim_red_data=hidden_representation.predict(data)
    print('data transformed')
    return dim_red_data

In [6]:
def evaluate_lstm(data_structure,timesteps,n_features,layer_size_1,layer_size_2,activation_func,optimizer_func,loss_func,n_epochs,n_batch_size,df_name,data):
    #create experiment Name to Load model 
    #experiment_name=df_name+'_'+data_structure+str(layer_size_1)+'_'+str(layer_size_2)+'_'+activation_func+'_'+optimizer_func+'_'+loss_func+'_'+str(n_epochs)+'_'+str(n_batch_size)
    experiment_name=df_name+'_'+data_structure+str(layer_size_1)+'_'+str(layer_size_2)+'_'+activation_func+'_'+optimizer_func+'_'+loss_func+'_'+str(n_epochs)+'_'+str(n_batch_size)


    
    model = keras.models.load_model('Cohort/LSTM_Models/'+experiment_name)
    score = model.evaluate(data,data)
    return score

In [19]:
#load experiments: 
a=pq.read_table('Cohort/Metrics_LSTM.parquet').to_pandas()
a=a.sort_values(by=['data_structure','loss'])
#a=
a.loc[a['timesteps']==7,'data_structure']='per_day'
a=a.sort_values(by=['data_structure','loss'])
a['validation_loss']=0
#load both datasets: 
data_per_patient ,data_sample_per_patient, df_name_per_patient, data_structure_per_patient=load_dataset(False,'cbow',20)
data_per_day ,data_sample_per_day, df_name_per_day, data_structure_per_day=load_dataset(True,'cbow',20)
for index, row in a.iterrows(): 
    if row['data_structure'] =='per_patient':
        data_structure=data_structure_per_patient
        data= data_sample_per_patient
        df_name= df_name_per_patient
    else : 
        data_structure=data_structure_per_day
        data= data_sample_per_day
        df_name= df_name_per_day
    timesteps=row['timesteps']
    n_features=row['n_features']
    layer_size_1=row['layer_size_1']
    layer_size_2=row['layer_size_2']
    activation_func=row['activation_func']
    optimizer_func=row['optimizer_func']
    loss_func=row['loss_func']
    n_epochs=row['n_epochs']
    n_batch_size=row['n_batch_size']
    if n_batch_size !=100:
        val_loss=evaluate_lstm(data_structure,timesteps,n_features,layer_size_1,layer_size_2,activation_func,optimizer_func,loss_func,n_epochs,n_batch_size,df_name,data)
        print(val_loss)
        a.loc[index, 'validation_loss']=val_loss[0]

Timeseries_per_patient_LSTM_Data_embedded_cbow_dim20_win5_mc0
Timeseries_per_patient_per_day_LSTM_Data_embedded_cbow_dim20_win5_mc0
354/354 [==============================] - 3s 9ms/step - loss: 0.2058 - mean_squared_error: 0.2058
[0.2057720422744751, 0.20577213168144226]
354/354 [==============================] - 3s 8ms/step - loss: 0.2080 - mean_squared_error: 0.2080
[0.20796135067939758, 0.20796138048171997]
354/354 [==============================] - 2s 7ms/step - loss: 0.2135 - mean_squared_error: 0.2135
[0.21352583169937134, 0.21352572739124298]
354/354 [==============================] - 3s 7ms/step - loss: 0.2853 - mean_squared_error: 0.2853
[0.2853085696697235, 0.2853086292743683]
354/354 [==============================] - 2s 5ms/step - loss: 0.7612 - mean_squared_error: 0.7612
[0.7611969113349915, 0.7611973285675049]
354/354 [==============================] - 2s 6ms/step - loss: 0.9086 - mean_squared_error: 0.9086
[0.908584713935852, 0.9085846543312073]
354/354 [===============

OSError: SavedModel file does not exist at: Cohort/LSTM_Models/Timeseries_per_patient_LSTM_Data_embedded_cbow_dim20_win5_mc0_per_patient32_16_tanh_adam_mse_3_1024/{saved_model.pbtxt|saved_model.pb}

In [21]:
a.to_parquet('Cohort/Metrics_LSTM_w_validation_loss.parquet')

In [24]:
a.sort_values(by=['validation_loss'])
a.to_excel("Cohort/LSTM_Models/11_05_Validation_loss.xlsx")

In [20]:
per_day=False
embedding_method='cbow'
embedding_size='20'
layer_size_1=32
layer_size_2=16
activation_func='tanh'
optimizer_func='adam'
loss_func='mse'
n_epochs=300
n_batch_size=1024

data,data_sample, df_name, data_structure= load_dataset(per_day,embedding_method,embedding_size)
timesteps=len(data[0])
n_features=len(data[0][0])
s=evaluate_lstm(data_structure,timesteps,n_features,layer_size_1,layer_size_2,activation_func,optimizer_func,loss_func,n_epochs,n_batch_size,df_name,data_sample)
print(s)

Timeseries_per_patient_LSTM_Data_embedded_cbow_dim20_win5_mc0
368/368 [==============================] - 14s 39ms/step - loss: 0.0418 - mean_squared_error: 0.0418
[0.04177021607756615, 0.04177020490169525]
